# In this notebook we are going to analyse the data scrapped from jumia.cm

In [1]:
# importing modules
import numpy as np
import pandas as pd
import re 
%matplotlib inline
import matplotlib.pyplot as plt
import time

In [4]:
# loading the scrapped data
df = pd.read_csv('./jumia_cm_smartphones_scraped_on_Sun_May_26_21:47:08_2019.csv', index_col=0)
print(df.info())
print(df['number-of-items-sold'])

<class 'pandas.core.frame.DataFrame'>
Index: 910 entries, Rise 31 Dual SIM 8Go HDD - Bleu + 1 Carte SIM et 1 Forfait Internet Offerts to Galaxy S9 Plus - 64Go HDD - 6Go RAM - 6,2" - Noir - 12 Mois
Data columns (total 12 columns):
seller                  910 non-null object
price                   910 non-null float64
number-of-items-sold    863 non-null float64
ram                     796 non-null float64
rom                     824 non-null float64
screen-size             768 non-null float64
color                   910 non-null object
front-camera            797 non-null float64
rear-camera             797 non-null float64
number_of_sims          910 non-null int64
date                    910 non-null object
os                      872 non-null object
dtypes: float64(7), int64(1), object(4)
memory usage: 92.4+ KB
None
name
Rise 31 Dual SIM 8Go HDD - Bleu + 1 Carte SIM et 1 Forfait Internet Offerts                              1001.0
Kodak Ektra - 5" - 32 Go HDD - 3 Go RAM - Noir     